# `std::variant`

The parameterized class `std::variant<A,B,...>` is an evolution of C unions. In addition, it memorizes the type of the last stored value, and raises exceptions if the developer makes type errors. From a "type algebra" point of view, we can think of it as the addition of types.

## Access when the current type is known

When the type of the variant's current value is known, there is no particular difficulty, `std::get` can be used :

In [1]:
#include <iostream>
#include <variant>

In [2]:
{
  std::variant<int, double> v, w ;
  int i, j ;
  double x, y ;
  
  v = 42 ;
  w = v ;
  i = std::get<int>(v) ;
  j = std::get<0>(w) ;
  
  w = 3.14 ;
  v = w ;
  x = std::get<double>(v) ;
  y = std::get<1>(w) ;
  
  // std::get<float>(v); // compilation error: no float in [int, double]
  // std::get<3>(v);     // compilation error: valid index values are 0 and 1
  // std::get<int>(v);   // runtime exception: v currently contains a double
  
  std::cout<<i<<" "<<j<<" "<<x<<" "<<y<<" "<<std::endl ;
}

42 42 3.14 3.14 


The advantage of `std::variant` compared to an union, is that a type mismatch between a write and a read is detected at the runtime.

## Access when the current type is unknown

If the type of the value currently stored in the variant is unknown, the `std::get_if` can be used instead :

In [3]:
#include <vector>

In [4]:
{
  using var_t = std::variant<int, double> ;
  std::vector<var_t> vals = { 42, 3.14 } ;
  for ( auto & val : vals )
   {
    int * ipval = std::get_if<int>(&val) ;
    double * dpval = std::get_if<double>(&val) ;
  
    if ( ipval ) std::cout << "int value: " << *ipval << std::endl ;     
    else if ( dpval ) std::cout << "double value: " << *dpval << std::endl ;    
    else throw "unknown value !" ;    
   }
}

int value: 42
double value: 3.14


## Polymorphic access

A `std::visit` can be combined with a polymorphic lambda function, which will infer the correct type and give direct access to the value:

In [5]:
{
  using var_t = std::variant<int, double> ;

  std::vector<var_t> vals = { 42, 3.14 } ;

  for ( auto val : vals )
    std::visit( []( auto arg )
     { std::cout<<"value: "<<arg<<std::endl ; } , val ) ; 
}

value: 42
value: 3.14


## Static differential treatment

If the treatment to be applied depends on the type of the current value, a "type pattern matching"  form can be set with a tad heavy syntax :

In [6]:
#include <type_traits>

In [7]:
{
  using var_t = std::variant<int, double> ;

  std::vector<var_t> vals = { 42, 3.14 } ;

  for ( auto val : vals )
    std::visit( []( auto arg )
     { 
      using T = std::decay_t<decltype(arg)> ;

      if constexpr (std::is_same_v<T, int>)
        std::cout<<"int value: "<<arg<<std::endl ;
      else if constexpr (std::is_same_v<T, double>)
        std::cout<<"double value: "<<arg<<std::endl ;

     },val) ;

}

int value: 42
double value: 3.14


## Exercise

In this example, replace inheritance with the use of `variant` :

In [8]:
%%file tmp.std-variant.cpp

#include <iostream>
#include <vector>

struct Particule
 {
  virtual void affiche() = 0 ;
  virtual ~Particule() = default ;
 } ;

struct Electron : public Particule
 { virtual void affiche(){ std::cout<<"E"<<std::endl ; } } ;

struct Proton : public Particule
 { virtual void affiche(){ std::cout<<"P"<<std::endl ; } } ;

struct Neutron : public Particule
 { virtual void affiche(){ std::cout<<"N"<<std::endl ; } } ;

int main()
 {
  std::vector<Particule *> ps =
   { new Electron, new Proton, new Neutron } ;

  for ( auto p : ps )
   { p->affiche() ; }
 }

Writing tmp.std-variant.cpp


In [9]:
!rm -f tmp.std-variant.exe && g++ -std=c++17 tmp.std-variant.cpp -o tmp.std-variant.exe

In [10]:
!./tmp.std-variant.exe

E
P
N


© *CNRS 2020*  
*Assembled and written in french by David Chamont, translated by Karim Asnaoui, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)